In [57]:
import time

import requests
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil import parser

In [58]:
fake = faker.Faker(locale='en')

In [59]:
headers = {
    'User-Agent': fake.chrome(),
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'referer': 'http://careers.google.com/jobs/results/'
}

In [60]:
URL = "http://careers.google.com/api/v3/search/"
VACANCY_URL = 'http://careers.google.com/jobs/results/'

In [61]:
search_params = {
    'page': 1,
    # 'company': 'Google', # One of Alphabet holding's company
    'q': '',
    'sort_by': 'date',
    'page_size': 100 # must be 20, 50, 75, 100
}

In [62]:
result = requests.get(URL, params=search_params, headers=headers)
result

<Response [200]>

In [63]:
total_count = result.json()['count']
max_pages = total_count // 100 - 1
print(total_count, max_pages)

7553 74


In [67]:
clear_string = lambda x: re.sub(' +', ' ', re.sub('<.*?>', '', x).replace('\n', '\n ')).strip()
title_to_url = lambda x: "-".join(re.sub('\s+', ' ', re.sub(r'[^a-zA-Z\s]', '', x.lower())).split(' '))

In [68]:
search_results = []
for page in tqdm(range(max_pages)):
    search_params['page'] = page

    result = requests.get(URL, params=search_params, headers=headers).json()
    found_jobs = result['jobs']

    if len(found_jobs) < 0:
        break

    for job_json in found_jobs:
        internal_id = job_json['id'].split('/')[-1]
        result_dict = {
            'title': job_json['title'],
            'internal_id': internal_id,
            'url': VACANCY_URL + internal_id + '-' + title_to_url(job_json['title']),
            'description': clear_string(job_json['description']),
            'responsibilities': clear_string(job_json['responsibilities']),
            'qualifications': clear_string(job_json['qualifications']),
            'company': job_json['company_name'],
            # 'country_code': job_json['country_code'],
            'publish_date': parser.parse(job_json['publish_date'])
        }
        search_results.append(result_dict)

    time.sleep(0.2) # sleep for 200 ms

100%|██████████| 74/74 [01:49<00:00,  1.49s/it]


In [69]:
snapshot = pd.DataFrame(search_results)
print(len(snapshot))
snapshot.sample(5)

5100


,title,internal_id,url,description,responsibilities,qualifications,company,publish_date
2070,"Product Operations Manager, Apps Quality, Goog...",138396433674838726,http://careers.google.com/jobs/results/1383964...,"As a Product Specialist, you will partner with...",Support the development and execution of the q...,Minimum qualifications:\n Bachelor's degree or...,Google,2022-05-31 16:31:53.195000+00:00
2710,"Audio Systems Engineer, Devices and Services",137517413588443846,http://careers.google.com/jobs/results/1375174...,"Our computational challenges are so big, compl...","Analyze, define, tune, validate, and debug aud...",Minimum qualifications:\n Bachelor's degree in...,Google,2022-05-24 04:02:23.304000+00:00
4485,"Account Manager, Google Operations Center",114438420849140422,http://careers.google.com/jobs/results/1144384...,gTech’s Strategy and Operations team (gSO) is ...,Provide business critical insights using analy...,Minimum qualifications:\n Bachelor's degree in...,Google,2022-04-25 18:32:10.298000+00:00
4222,Senior Data Center Real Estate Researcher,74996363927921350,http://careers.google.com/jobs/results/7499636...,A problem isn’t truly solved until it’s solved...,"Gather, discern, and collate market intelligen...",Minimum qualifications:\n Bachelor’s degree in...,Google,2022-04-28 14:02:08.583000+00:00
2866,"Partner Customer Engineer, Data Analytics, Tec...",116710526488060614,http://careers.google.com/jobs/results/1167105...,The Google Cloud Platform team helps customers...,Help Sales and Google Cloud Architect teams un...,Minimum qualifications:\n Bachelor's degree in...,Google,2022-05-20 13:31:58.640000+00:00


In [72]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5100 entries, 0 to 5099
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype                  
---  ------            --------------  -----                  
 0   title             5100 non-null   object                 
 1   internal_id       5100 non-null   object                 
 2   url               5100 non-null   object                 
 3   description       5100 non-null   object                 
 4   responsibilities  5100 non-null   object                 
 5   qualifications    5100 non-null   object                 
 6   company           5100 non-null   object                 
 7   publish_date      5100 non-null   datetime64[ns, tzutc()]
dtypes: datetime64[ns, tzutc()](1), object(7)
memory usage: 318.9+ KB


In [73]:
snapshot['company'].value_counts()

Google     4760
YouTube     253
Fitbit       87
Name: company, dtype: int64

In [74]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'17-06-2022'

In [75]:
snapshot.to_csv(f'../data/google/{current_date}.csv')